#### 使用开源项目tensorflow/nmt

In [7]:
# 下载项目
!git clone https://github.com/tensorflow/nmt/
!ls

fatal: destination path 'nmt' already exists and is not an empty directory.
README.md             nmt                   seq2seq-chatbot.ipynb


**我们下载小黄鸡语料，并对它做一个处理，使得它符合seq2seq模型的输入格式**

In [10]:
!wget https://github.com/candlewill/Dialog_Corpus/raw/master/xiaohuangji50w_nofenci.conv.zip
!unzip xiaohuangji50w_nofenci.conv.zip

--2019-12-02 10:29:01--  https://github.com/candlewill/Dialog_Corpus/raw/master/xiaohuangji50w_nofenci.conv.zip
正在解析主机 github.com (github.com)... 192.30.253.112
正在连接 github.com (github.com)|192.30.253.112|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 302 Found
位置：https://raw.githubusercontent.com/candlewill/Dialog_Corpus/master/xiaohuangji50w_nofenci.conv.zip [跟随至新的 URL]
--2019-12-02 10:29:02--  https://raw.githubusercontent.com/candlewill/Dialog_Corpus/master/xiaohuangji50w_nofenci.conv.zip
正在解析主机 raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.76.133
正在连接 raw.githubusercontent.com (raw.githubusercontent.com)|151.101.76.133|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：10232473 (9.8M) [application/zip]
正在保存至: “xiaohuangji50w_nofenci.conv.zip”

xiaohuangji50w_nofe 100%[===================>]   9.76M  5.80MB/s  用时 1.7s      

2019-12-02 10:29:05 (5.80 MB/s) - 已保存 “xiaohuangji50w_nofenci.conv.zip” [10232473/10232473])

Archive:  xiaohuangji50w_nofenci.conv.zip
  inflating: xiaohuangj

In [19]:
# 把第一行的E和换行换后的第二行换成Q M
!perl -pi.bak -e 's/(E\s)/\1Q /g' xiaohuangji50w_nofenci.conv
# 把第二行的(Q, M)换成 Q
!perl -pi.bak -e 's/(Q M)/Q/g' xiaohuangji50w_nofenci.conv
# 第三行的M换成 A
!perl -pi.bak -e 's/(M )/A /g' xiaohuangji50w_nofenci.conv

In [20]:
!head -30 xiaohuangji50w_nofenci.conv

E
Q 呵呵
A 是王若猫的。
E
Q 不是
A 那是什么？
E
Q 怎么了
A 我很难过，安慰我~
E
Q 开心点哈,一切都会好起来
A 嗯 会的
E
Q 我还喜欢她,怎么办
A 我帮你告诉她？发短信还是打电话？
E
Q 短信
A 嗯嗯。我也相信
E
Q 你知道谁么
A 肯定不是我，是阮德培
E
Q 许兵是谁
A 吴院四班小帅哥
E
Q 这么假
A 三鹿奶粉也假，不一样的卖啊
E
Q 许兵是傻逼
A 被你发现了。


In [24]:
text = open('xiaohuangji50w_nofenci.conv').read().split('E\n')
text[1]

'Q 呵呵\nA 是王若猫的。\n'

**分词**

In [30]:
import jieba
def split_conv(in_f, out_q, out_a):
    out_question = open(out_q, 'w')
    out_answer = open(out_a, 'w')
    text = open(in_f).read().split('E\n')
    for pair in text:
        # 句子长度太短的对话，就过滤掉，跳过
        if len(pair) <= 4:
            continue
        # 切分问题和回答
        contents = pair.split('\n')
        out_question.write(' '.join(jieba.lcut(contents[0].strip('Q '))) + '\n')
        out_answer.write(' '.join(jieba.lcut(contents[1].strip('A '))) + '\n')
    out_question.close()
    out_answer.close()

In [31]:
in_f = 'xiaohuangji50w_nofenci.conv'
out_q = 'question.file'
out_a = 'answer.file'
split_conv(in_f, out_q, out_a)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/g_/rv2sg65j1_g2znz05v_snmth0000gn/T/jieba.cache
Loading model cost 0.652 seconds.
Prefix dict has been built succesfully.


In [32]:
!head -10 question.file

呵呵
不是
怎么 了
开心 点哈 , 一切 都 会 好 起来
我 还 喜欢 她 , 怎么办
短信
你 知道 谁 么
许兵 是 谁
这么 假
许兵 是 傻 逼
